# 🧠 Deep Hedging avec PyTorch

Ce notebook simule un marché Black-Scholes et apprend une stratégie de couverture d'option européenne (call) à l'aide d'un réseau de neurones (MLP).

## 1. Imports

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'torch'

## 2. Simulation de trajectoires de marché (modèle Black-Scholes)

In [ ]:
# Fonction de simulation Black-Scholes
def simulate_black_scholes_paths(S0, r, sigma, T, N, M, seed=42):
    np.random.seed(seed)
    dt = T / N
    paths = np.zeros((M, N + 1))
    paths[:, 0] = S0
    for t in range(1, N + 1):
        Z = np.random.randn(M)
        paths[:, t] = paths[:, t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
    return paths

## 3. Modèle de couverture (réseau neuronal simple)

In [ ]:
# Modèle PyTorch
class HedgingNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.model(x)

## 4. Delta classique de Black-Scholes (pour comparaison)

In [ ]:
# Formule delta call européen

def bs_delta_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

## 5. Entraînement du modèle et calcul des erreurs

In [ ]:
# Fonction d'entraînement
def train_hedging_model(S_paths, K, r, T, sigma, model, n_epochs=30, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    N = S_paths.shape[1] - 1
    dt = T / N

    train_losses = []
    nn_residuals = []
    delta_residuals = []

    for epoch in range(n_epochs):
        total_loss = 0
        for i in range(S_paths.shape[0]):
            S = S_paths[i]
            S_tensor = torch.tensor(S[:-1], dtype=torch.float32).view(-1, 1).to(device)
            dS = torch.tensor(S[1:] - S[:-1], dtype=torch.float32).view(-1, 1).to(device)

            phi = model(S_tensor)
            pnl = torch.sum(phi * dS)
            payoff = max(S[-1] - K, 0.0)
            payoff_tensor = torch.tensor(payoff, dtype=torch.float32).to(device)

            loss = loss_fn(pnl, payoff_tensor)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            if epoch == n_epochs - 1:
                nn_residuals.append((pnl.item() - payoff))
                T_grid = np.linspace(0, T, N)
                delta_positions = bs_delta_call(S[:-1], K, T - T_grid, r, sigma)
                pnl_delta = np.sum(delta_positions * (S[1:] - S[:-1]))
                delta_residuals.append(pnl_delta - payoff)

        train_losses.append(total_loss / S_paths.shape[0])
        print(f"Epoch {epoch + 1}/{n_epochs} - Loss: {train_losses[-1]:.4f}")

    return train_losses, nn_residuals, delta_residuals

## 6. Paramètres du problème et exécution

In [ ]:
S0 = 100
r = 0.01
sigma = 0.2
T = 1.0
N = 30
M = 1000
K = 100

S_paths = simulate_black_scholes_paths(S0, r, sigma, T, N, M)
model = HedgingNN(input_dim=1, hidden_dim=16)

train_losses, nn_residuals, delta_residuals = train_hedging_model(S_paths, K, r, T, sigma, model, n_epochs=30, lr=1e-3)

## 7. Visualisation des résultats

In [ ]:
# Courbe de perte
plt.figure(figsize=(8, 4))
plt.plot(train_losses)
plt.title("Courbe de perte pendant l'entraînement")
plt.xlabel("Époque")
plt.ylabel("Loss (MSE)")
plt.grid(True)
plt.show()

In [ ]:
# Histogrammes des résidus
plt.figure(figsize=(10, 4))
plt.hist(nn_residuals, bins=40, alpha=0.5, label="Deep Hedging")
plt.hist(delta_residuals, bins=40, alpha=0.5, label="Delta classique")
plt.axvline(0, color='black', linestyle='--')
plt.title("Distribution des erreurs de couverture")
plt.xlabel("Résidu (P&L - Payoff)")
plt.ylabel("Fréquence")
plt.legend()
plt.grid(True)
plt.show()

## 8. Métrique quantitative : RMSE des stratégies

In [ ]:
rmse_nn = mean_squared_error([0]*len(nn_residuals), nn_residuals, squared=False)
rmse_delta = mean_squared_error([0]*len(delta_residuals), delta_residuals, squared=False)

print(f"RMSE Deep Hedging : {rmse_nn:.4f}")
print(f"RMSE Delta classique : {rmse_delta:.4f}")